In [1]:

import os
import sys
import random
import logging
import rasterio
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import segmentation_models_pytorch as smp

path_cur = os.path.abspath(os.getcwd())

sys.path.append(path_cur)

from vims_dataloader_backbone import GenDEBRIS

sys.path.append(os.path.join(up(up(path_cur)), 'utils'))

from vims_metrics import Evaluation, confusion_matrix
from vims_assets import labels
from pathlib import Path

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

root_path = up(up(path_cur))



In [2]:
path_cur

'/rapids/notebooks/sciclone/geograd/Miranda/github/shoreline_structure/semantic_segmentation/unet'

In [3]:
def main(options):
    # Transformations
    
    transform_test = transforms.Compose([transforms.ToTensor()])
    standardization = None # transforms.Normalize(bands_mean, bands_std)
    
    # Construct Data loader

    dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization, data_name = options["data_source"])

    test_loader = DataLoader(   dataset_test, 
                                batch_size = options['batch'], 
                                shuffle = False)
    
    global labels
    # Aggregate Distribution Mixed Water, Wakes, Cloud Shadows, Waves with Marine Water

#     # Use gpu or cpu
#     if torch.cuda.is_available():
#         device = torch.device("cuda")
#     else:
#         device = torch.device("cpu")
    
    device = torch.device("cpu")
    
    model = smp.Unet(options["backbone"],
                     encoder_weights="imagenet",
                     in_channels=options["input_channels"],
                     classes=options["output_channels"],
                    activation='softmax')
        
    
#     model = UNet(input_bands = options['input_channels'], 
#                  output_classes = options['output_channels'], 
#                  hidden_channels = options['hidden_channels'])

    model.to(device)

    # Load model from specific epoch to continue the training or start the evaluation
    model_file = options['model_path']
    logging.info('Loading model files from folder: %s' % model_file)

    checkpoint = torch.load(model_file, map_location = device)
    model.load_state_dict(checkpoint)

    del checkpoint  # dereference
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model.eval()

    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        
        for (batch_idx, batch_val) in enumerate(tqdm(test_loader, desc="testing")):

            image = batch_val['image'].to(device)
            target = batch_val['mask'].to(device)

            logits = model(image)

            # Accuracy metrics only on annotated pixels
            logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
            logits = logits.reshape((-1,options['output_channels']))
            target = target.reshape(-1)
            mask = target != -1
            logits = logits[mask]
            target = target[mask]
            
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            target = target.cpu().numpy()
            
            y_predicted += probs.argmax(1).tolist()
            y_true += target.tolist()
        
        ####################################################################
        # Save Scores to the .log file                                     #
        ####################################################################
        acc = Evaluation(y_predicted, y_true)
        logging.info("\n")
        logging.info("STATISTICS: \n")
        logging.info("Evaluation: " + str(acc))
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        conf_mat = confusion_matrix(y_true, y_predicted, labels)
        
        logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        
        if options['predict_masks']:
            
            path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
            ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

            impute_nan = np.tile(bands_mean, (256,256,1))
                        
            for roi in tqdm(ROIs):
            
#                 roi_folder = '_'.join(['S2'] + roi.split('_')[:-1])             # Get Folder Name
#                 roi_name = '_'.join(['S2'] + roi.split('_'))                    # Get File Name
#                 roi_file = os.path.join(path, roi_folder,roi_name + '.tif')     # Get File path
                
                roi_file = os.path.join(path, roi)
                os.makedirs(options['gen_masks_path'], exist_ok=True)
            
                output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')
            
                # Read metadata of the initial image
                with rasterio.open(roi_file, mode ='r') as src:
                    tags = src.tags().copy()
                    meta = src.meta
                    image = src.read()
                    image = np.moveaxis(image, (0, 1, 2), (2, 0, 1))
                    dtype = src.read(1).dtype
            
                # Update meta to reflect the number of layers
                meta.update(count = 1)
            
                # Write it
                with rasterio.open(output_image, 'w', **meta) as dst:
                    
                    # Preprocessing before prediction
                    nan_mask = np.isnan(image)
                    image[nan_mask] = impute_nan[nan_mask]
            
                    image = transform_test(image)
                    
                    image = standardization(image)
                    
                    # Image to Cuda if exist
                    image = image.to(device)
            
                    # Predictions
                    logits = model(image.unsqueeze(0))
            
                    probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()
            
                    probs = probs.argmax(1).squeeze()+1
                    
                    # Write the mask with georeference
                    dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
                    dst.update_tags(**tags)



In [21]:

data_name = 'Image_allyear_VA_256'

# # Pixel-Level class distribution for each dataset
# """
# {'Image_allyear_merged_512': array([0.49019898, 0.44645175, 0.02657669, 0.03677258]), 
#  'Image_after_2010_merged_512': array([0.45526231, 0.43276168, 0.05267328, 0.05930273]), 
#  'Image_after_2010_merged_256': array([0.32490837, 0.41855632, 0.17178225, 0.08475305]), done
#  'Image_allyear_merged_256': array([0.44056167, 0.41559786, 0.09143975, 0.05240071]), done
#  'Image_allyear_merged_1024': array([0.50426896, 0.4565353 , 0.01033589, 0.02885985]),
#  'Image_after_2010_merged_1024': array([0.49582348, 0.43486081, 0.02433212, 0.04498359]),
#  'Image_after_2010_VA_512': array([0.47100772, 0.44772889, 0.01990966, 0.06135373]), 
#  'Image_after_2010_VA_256': array([0.37283283, 0.48029399, 0.04961892, 0.09725425]), done
#  'Image_allyear_VA_512': array([0.49843776, 0.45395527, 0.01021636, 0.03739061]), 
#  'Image_allyear_VA_256': array([0.47332474, 0.44650446, 0.02387324, 0.05629757])} 
# """

In [22]:
data_source = data_name
lr = 2e-3
backbone = 'resnet101'

model_name = 'unet_{}_{}_{}'.format(backbone, lr, data_source)
model_epoch = '100'

model_path = os.path.join(up(path_cur), model_name, model_epoch, 'model.pth')
gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'VABP', 'paper', 'test_masks')


options = {'batch': 32, 'backbone': backbone, 'data_source': data_source,
          'input_channels': 3, 'output_channels': 4,
          'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path}

In [23]:
main(options)

Load test set to memory:   8%|▊         | 19/225 [00:00<00:01, 186.33it/s]

/rapids/notebooks/sciclone/geograd/Miranda/github/shoreline_structure/datasets/Image_allyear_VA_256


testing:   0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/envs/vims/lib/python3.7/site-packages/segmentation_models_pytorch/base/modules.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)
testing: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


Evaluation: {'macroPrec': 0.6042316389104878, 'microPrec': 0.6756581412366441, 'weightPrec': 0.6778333923719474, 'macroRec': 0.6120106538418562, 'microRec': 0.6756581412366441, 'weightRec': 0.6756581412366441, 'macroF1': 0.607155289060172, 'microF1': 0.6756581412366441, 'weightF1': 0.6765190114539078, 'subsetAcc': 0.6756581412366441, 'IoU': 0.44268429093756945}
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             281937.0  105064.0   1422.0     2585.0   391008.0   0.72
Rip Rap                           93342.0  214759.0  12371.0    10133.0   330605.0   0.65
Groin                                 4.0    6322.0  11756.0     5295.0    23377.0    0.5
Breakwater                         6536.0   13041.0   1849.0    28930.0    50356.0   0.57
Sum                              381819.0  339186.0  27398.0    46943.0       mPA:   0.61
IoU                                  0.57      0.47      0.3       0.42     